<b> exploring and clustering the neighborhoods in toronto </b>

> we will scrap "https://en.wikipedia.org/wiki/:_M" using the beatuifulsoup lib as suggested in the course

In [1]:
import pandas as pd 
import numpy as np
import requests 
import bs4 as bs
import urllib.request
import lxml.html as lh 


> a url link to access the wiki page with the tabel 

In [11]:
url   = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"


> bs4 (beautifulsoup4) is used to scrap the tabel content from the wiki page for toranto city 

In [12]:
def get_tabel(n,c):
    page = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(page,"lxml")
    table = soup.find("table" , class_ = n)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]# th is html tag that define the header cell
    data = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
          for tr in table.find_all("tr")]    
    data = [row for row in data if len(row) == c]
    df = pd.DataFrame(data , columns = header)
    return df

In [13]:
toronto_postal = get_tabel("wikitable" , 3)
toronto_postal.describe()


,Postcode,Borough,Neighbourhood
count,287,287,287
unique,180,12,208
top,M8Y,Not assigned,Not assigned
freq,8,77,78


> Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. for example postcode M8Y will be droped from our final dataframe

> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
we will be using the "isin" to check if the postcode is not assaigned.

> when a postal code appears in more than one neighborhood, those nieghborhoods will be grouped by the POSTCODE and Borough and sperarated by a comma in the neigborhood column 

In [14]:
toronto_postal_new = toronto_postal[~toronto_postal["Borough"].isin(["Not assigned"])]
toronto_postal_new = toronto_postal_new.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
toronto_postal_new.loc[toronto_postal_new["Neighbourhood"] == "Not assigned" , ["Neighbourhood"]] = toronto_postal_new["Borough"]
toronto = toronto_postal_new.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [13]:
toronto.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [15]:
toronto.shape

(103, 3)

> now we'll creat a csv file that contains the cleand up dataframe 

In [16]:
toronto.to_csv('Toronto_task1.csv',index=False)